In [1]:
import pandas as pd
import numpy as np

import time
import datetime

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import scale

In [2]:
features = pd.read_csv('features/features.csv', index_col='match_id')
features.drop('duration radiant_win tower_status_radiant tower_status_dire barracks_status_radiant barracks_status_dire'.split(' '), inplace=True, axis=1)

features_test = pd.read_csv('features_test/features_test.csv', index_col='match_id')

In [10]:
X_train , Y_train = features.fillna(0).as_matrix().astype(float), pd.read_csv('features/features.csv', index_col='match_id')['radiant_win'].fillna(0).as_matrix().astype(float)
X_test = features_test.fillna(0).as_matrix().astype(float)

In [4]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

1.Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [19]:
def testing_grid(X_train, Y_train, c=[10.0 ** i for i in range(-9, 8)]):
    X_train = scale(X_train)
    
    grid = {'C': c}
    clf = LogisticRegression(penalty='l2')
    
    start_time = datetime.datetime.now()
    gs = GridSearchCV(clf, scoring='roc_auc', param_grid=grid, cv=cv, return_train_score=True)
    gs.fit(X_train,Y_train)
    print ('Time elapsed:', datetime.datetime.now() - start_time)
    
    means = gs.cv_results_['mean_test_score']
    stds = gs.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, gs.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    return c[np.argmax(means)]
    
def testing_cros(X_train, Y_train, c=[10.0 ** i for i in range(-9, 8)]):     
    start_time = datetime.datetime.now()
    scores=[]
    X_train = scale(X_train)
    for x in c:
        print (x)
        scores.append(np.array(cross_val_score(LogisticRegression(C=x), X_train, Y_train, scoring='roc_auc', cv=cv)))
    scores=np.array(scores)
    print ('Time elapsed:', datetime.datetime.now() - start_time)
    print (c[np.argmax([np.mean(sc) for sc in scores])])
    return c[np.argmax([np.mean(sc) for sc in scores])]

In [20]:
testing_grid(scale(X_train), Y_train)

C:\Users\Denis\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


Time elapsed: 0:02:47.643152
0.687 (+/-0.003) for {'C': 1e-09}
0.687 (+/-0.003) for {'C': 1e-08}
0.687 (+/-0.003) for {'C': 1e-07}
0.688 (+/-0.003) for {'C': 1e-06}
0.695 (+/-0.003) for {'C': 1e-05}
0.711 (+/-0.003) for {'C': 0.0001}
0.716 (+/-0.003) for {'C': 0.001}
0.717 (+/-0.002) for {'C': 0.01}
0.717 (+/-0.002) for {'C': 0.1}
0.717 (+/-0.002) for {'C': 1.0}
0.717 (+/-0.002) for {'C': 10.0}
0.717 (+/-0.002) for {'C': 100.0}
0.717 (+/-0.002) for {'C': 1000.0}
0.717 (+/-0.002) for {'C': 10000.0}
0.717 (+/-0.002) for {'C': 100000.0}
0.717 (+/-0.002) for {'C': 1000000.0}
0.717 (+/-0.002) for {'C': 10000000.0}


0.01